<a href="https://colab.research.google.com/github/IsraelAbebe/ml-algorithems-from-scratch/blob/master/algorithms/neural-net/Neural_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!unzip drive/My\ Drive/data/data.zip 
# ls

Archive:  drive/My Drive/data/data.zip
   creating: data/
   creating: data/mnist/
   creating: data/mnist/raw/
  inflating: data/mnist/raw/train-images-idx3-ubyte  
  inflating: data/mnist/raw/train-labels-idx1-ubyte  
  inflating: data/mnist/raw/t10k-images-idx3-ubyte  
  inflating: data/mnist/raw/t10k-labels-idx1-ubyte  
   creating: data/mnist/processed/
  inflating: data/mnist/processed/training.pt  
  inflating: data/mnist/processed/test.pt  
   creating: data/mnist/MNIST/
   creating: data/mnist/MNIST/raw/
  inflating: data/mnist/MNIST/raw/train-images-idx3-ubyte  
  inflating: data/mnist/MNIST/raw/train-labels-idx1-ubyte  
  inflating: data/mnist/MNIST/raw/t10k-images-idx3-ubyte  
  inflating: data/mnist/MNIST/raw/t10k-labels-idx1-ubyte  
   creating: data/mnist/MNIST/processed/
  inflating: data/mnist/MNIST/processed/training.pt  
  inflating: data/mnist/MNIST/processed/test.pt  
   creating: data/cifar10/
  inflating: data/cifar10/cifar-10-python.tar.gz  
   creating: data/ci

In [0]:
import torch
# TO LOAD MNIST DATASET
from dlc_practical_prologue import *

In [5]:
data = load_data(one_hot_labels=True,normalize=True)

* Using MNIST


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:53: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:43: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


** Reduce the data-set (use --full for the full thing)
** Use 1000 train and 1000 test samples


\begin{aligned}
&\tanh (x)
\\~\\
&\frac{d \tanh (x)}{d x}
\\~\\
&\|t-v\|^{2}
\\~\\
&2 *(t-v)
\end{aligned}


In [0]:
# HELPER FUNCTIONS
def sigma(inputs):
    return torch.tanh(inputs)

def dsigma(inputs):
    return 1-torch.pow(sigma(inputs),2)

def loss(v,t):
    return torch.pow((v-t),2).sum()

def dloss(v,t):
    return 2*(v-t)


In [0]:
def get_params(train_images,hidden_dim=50,out_dim=10,std = 1e-6):
    w1 = torch.empty((hidden_dim,train_images.size(1))).normal_(0,std)
    b1 = torch.empty((hidden_dim,1)).normal_(0,std)

    w2 = torch.empty((out_dim,hidden_dim)).normal_(0,std)
    b2 = torch.empty((out_dim,1)).normal_(0,std)

    dl_w1 = torch.empty_like(w1)
    dl_b1 = torch.empty_like(b1)

    dl_w2 = torch.empty_like(w2)
    dl_b2 = torch.empty_like(b2)

    return w1,b1,w2,b2,dl_w1,dl_b1,dl_w2,dl_b2
  
  
def forward_pass(w1,b1,w2,b2,x):
    s1 = torch.add(torch.mm(w1,x),b1)
    x1 = sigma(s1)

    s2 = torch.add(torch.mm(w2,x1),b2)
    x2 = sigma(s2)

    return x,s1,x1,s2,x2

    
    
def backward_pass(w1,b1,w2,b2,t,x,s1,x1,s2,x2,dl_dw1,dl_db1,dl_dw2,dl_db2):
    dl_dx2 = dloss(x2,t)
    dl_ds2 = dl_dx2*dsigma(s2)

    dl_dw2.add_(torch.mm(dl_ds2,x1.view(1,-1)))
    dl_db2.add_(dl_ds2)

    dl_dx1 = torch.mm(w2.T,dl_ds2)
    dl_ds1 = dl_dx1*dsigma(s1)

    dl_dw1.add_(torch.mm(dl_ds1,x.view(1,-1)))
    dl_db1.add_(dl_ds1)

In [8]:
train_images,train_labels,test_images,test_labels = data

w1,b1,w2,b2,dl_dw1,dl_db1,dl_dw2,dl_db2 = get_params(train_images,hidden_dim=50,out_dim=10)

train_labels,test_labels = train_labels * 0.9,test_labels * 0.9
train_images,train_labels = train_images * 0.9,train_labels * 0.9

train_size = len(train_images)
test_size = len(test_images)

lr = 1e-1 / train_size
epoch = 1000

for i in range(epoch):
    dl_dw1.zero_()
    dl_db1.zero_()
    dl_dw2.zero_()
    dl_db2.zero_()

    correct_count = 0
    test_correct_count = 0

    for image,labels in zip(train_images,train_labels):
        x,s1,x1,s2,x2 = forward_pass(w1,b1,w2,b2,image.view(-1,1))
        backward_pass(w1,b1,w2,b2,labels.view(-1,1),image.view(-1,1),s1,x1,s2,x2,dl_dw1,dl_db1,dl_dw2,dl_db2)

    w1 = w1 -  lr * dl_dw1
    b1 = b1 -  lr * dl_db1
    w2 = w2 -  lr * dl_dw2
    b2 = b2 -  lr * dl_db2

    for img,lbl in zip(train_images,train_labels):
        _,_,_,_,x2 = forward_pass(w1,b1,w2,b2,img.view(-1,1))
        if x2.max(0)[1].item()==lbl.max(0)[1].item():
            correct_count +=1

    for t_img,t_lbl in zip(test_images,test_labels):
        _,_,_,_,t_x2 = forward_pass(w1,b1,w2,b2,t_img.view(-1,1))
        if t_x2.max(0)[1].item()==t_lbl.view(-1,1).max(0)[1].item():
            test_correct_count +=1
    
    if i%100 == 0:
        print('Epoch: {}, Train Accuracy(Error Rate): {:.02f}({:.02f}) , Test Accuracy(Error Rate): {:.02f}({:.02f})'.format(i,\
            correct_count/train_size,1-(correct_count/train_size),test_correct_count/test_size,1-(test_correct_count/test_size)))

Epoch: 0, Train Accuracy(Error Rate): 0.12(0.88) , Test Accuracy(Error Rate): 0.10(0.90)
Epoch: 100, Train Accuracy(Error Rate): 0.78(0.22) , Test Accuracy(Error Rate): 0.72(0.28)
Epoch: 200, Train Accuracy(Error Rate): 0.91(0.09) , Test Accuracy(Error Rate): 0.80(0.20)
Epoch: 300, Train Accuracy(Error Rate): 0.95(0.05) , Test Accuracy(Error Rate): 0.83(0.17)
Epoch: 400, Train Accuracy(Error Rate): 0.96(0.04) , Test Accuracy(Error Rate): 0.82(0.18)
Epoch: 500, Train Accuracy(Error Rate): 0.97(0.03) , Test Accuracy(Error Rate): 0.85(0.15)
Epoch: 600, Train Accuracy(Error Rate): 0.98(0.02) , Test Accuracy(Error Rate): 0.84(0.16)
Epoch: 700, Train Accuracy(Error Rate): 0.99(0.01) , Test Accuracy(Error Rate): 0.85(0.15)
Epoch: 800, Train Accuracy(Error Rate): 0.99(0.01) , Test Accuracy(Error Rate): 0.84(0.16)
Epoch: 900, Train Accuracy(Error Rate): 0.99(0.01) , Test Accuracy(Error Rate): 0.85(0.15)
